## Insight from the various information

Many sales person want to know about their counter accounts. And they also want to know the business trend and insights to deliver more relevant solutions to their customers. 

But in many cases, the customers - especially greenfield customers - doesn't want to show their pains and short-term/long-term goals to the sales person in the vendors. 

Our goal is to analyze their business status from their financial statements and pain points from ther crapped news from the internet.


In [ ]:
#!pip install google-cloud-aiplatform langchain chroma
#!pip install html5lib
#!pip install openai
#!pip install opendartreader

In [13]:
import os
import OpenDartReader
from OpenDartReader.dart_list import *

DART_API_KEY=os.getenv("DART_API_KEY")
dartReader = OpenDartReader(DART_API_KEY)
company_list = corp_codes(DART_API_KEY)

In [ ]:
import dart_fss as dart
import os

# Open DART API KEY 설정
api_key=os.getenv('DART_API_KEY')
dart.set_api_key(api_key=api_key)
dart.get_corp_list()

In [17]:
import vertexai
from langchain.llms import VertexAI

PROJECT_ID = os.getenv("PROJECT_NUMBER")  # @param {type:"string"}
vertexai.init(project=PROJECT_ID, location="us-central1")

llm = VertexAI(
    model_name="text-bison@latest",
    max_output_tokens=2048,
    temperature=0,
    top_p=0.8,
    top_k=40,
)


In [18]:
from langchain.tools import Tool
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()

def top5_results(query):
    return search.results(query=query, num_results=5)

search_tool = Tool(
    name="search",
    description="search snippets through google internet search engine with result format json - title, link, snippet - list and 5 results.",
    func=top5_results,
)


[{'title': '삼성전자', 'link': 'https://www.samsung.com/sec/', 'snippet': '삼성위크 영어로 되어 있는 문자 주위로 티비와 사랑을 나누는 캐릭터,. Galaxy Z Fold5 I Z Flip5 I'}, {'title': '삼성전자 뉴스룸', 'link': 'https://news.samsung.com/kr/', 'snippet': "Samsung Newsroom Korea – 삼성전자 뉴스룸은 삼성전자의 대표 뉴스 채널입니다. · 삼성전자, 2023년 3분기 실적 발표 · 삼성전자, '갤럭시 Z 플립5 레트로' 출시 · [삼성\xa0..."}, {'title': 'Samsung US | Mobile | TV | Home Electronics | Home Appliances ...', 'link': 'https://www.samsung.com/us/', 'snippet': 'Discover the latest in electronic & smart appliance technology with Samsung. Find the next big thing from smartphones & tablets to laptops & tvs & more.'}, {'title': '삼성전자서비스', 'link': 'https://www.samsungsvc.co.kr/', 'snippet': '서비스센터 정보(위치, 수리제품, 영업시간), 출장 서비스 신청, 제품 상담(챗봇/이메일/전화/원격/수어 상담), 동영상 가이드, 다운로드 등 다양한 컨텐츠를 제공하고\xa0...'}, {'title': '삼성전자 - 나무위키', 'link': 'https://namu.wiki/w/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90', 'snippet': '1. 개요[편집] ... 삼성전자(SAMSUNG ELECTRONICS)는 삼성그룹의 계열사로 대한민국에 본사를 두고 전자제품을 제조하는 대한민국 최대의 다국적 기업이다. 

In [21]:
from bs4 import BeautifulSoup
import requests


def getHtmlContentFromUrl(urllink):
  response = requests.get(urllink, timeout=10)
  html_content = response.text
  soup = BeautifulSoup(html_content, 'lxml')
  for script in soup.find_all('script'):
    script.extract()
  list_content = soup.find_all('p')
  joined_content = ''
  for content in list_content:
    joined_content += content.text
  return joined_content

crawl_tool = Tool(
  name="crawl",
  description="crawl html content from url",
  func=getHtmlContentFromUrl,
)

In [25]:
from pydantic import BaseModel, Field
from langchain.agents import AgentType, initialize_agent

tools = [search_tool, crawl_tool]

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [26]:
agent.run("Summarize the Samsung Electronics 2020 annual report.")



> Entering new AgentExecutor chain...
 The Samsung Electronics 2020 annual report is a comprehensive document that provides an overview of the company's financial performance, business segments, and future outlook.
Action: crawl
Action Input: https://www.samsung.com/sec/ir/financial-information/annual-report/
Observation: 갤럭시 
Z 플립5 레트로삼성위크 슈퍼세일시간 맞춤 설치 서비스삼성케어플러스삼성 강남삼닷 Live갤럭시 캠퍼스 스마트싱스e식품관멤버십지속가능경영삼성전자 앱BESPOKE 냉장고 인피니트
검색 결과0 개 
		괜찮으시다면 가까운 곳에서다시 한번 페이지를 찾아볼까요?
삼성전자 주식회사 대표이사 : 한종희
사업자등록번호 : 124-81-00998
사업자 정보확인
통신판매업 신고 : 2000-경기수원-0515

사업장주소 : 경기도 수원시 영통구 삼성로 129(매탄동)
호스팅서비스사업자 : 한국아이비엠(주)
대표번호 : 02-2255-0114
제품/서비스/멤버십: 1588-3366(통화요금 : 발신자부담)
구매문의 : 1588-6084
e식품관 문의 : 1811-9228
본 사이트에서 판매되는 상품 중에는 등록된 개별 판매자가 판매하는 상품이 포함되어 있습니다. 개별 판매자 판매 상품의 경우 삼성전자(주)는 통신판매중개업자로서 통신판매의 당사자가 아니므로, 개별 판매자가 등록한 상품, 거래정보 및 거래 등에 대해 책임을 지지 않습니다.본 사이트의 컨텐츠는 저작권법의 보호를 받는 바  무단 전재, 복사, 배포 등을 금합니다.Copyright © 1995-2023 Samsung. All Rights Reserved.공유본 웹사이트는 게시된 이메일 주소가 전자우편 수집 프로그램이나 그 밖의 기술적 장치

KeyboardInterrupt: 

In [9]:
def searchSimilarStringInDataframeColumns(df_companylist, keyword):
    return df_companylist[df_companylist['corp_name'].str.startswith(keyword)]

# def searchDocumentsFromDart(documentId):
#   return dart.list(documentId, start='1999-01-01', kind='A') 

#print(searchSimilarStringInDataframeColumns(company_list, company_name))

In [37]:
company_name = "CJ대한통운"

In [14]:
def getCompanyOverallInformation(corp_code):
  return dartReader.company(corp_code)

print(getCompanyOverallInformation("00113410"))

{'status': '000', 'message': '정상', 'corp_code': '00113410', 'corp_name': '씨제이대한통운(주)', 'corp_name_eng': 'CJ Logistics Corporation ', 'stock_name': 'CJ대한통운', 'stock_code': '000120', 'ceo_nm': '강신호,민영학(각자대표)', 'corp_cls': 'Y', 'jurir_no': '1101110006167', 'bizr_no': '1108105034', 'adres': '서울특별시 종로구 종로5길 7(청진동) Tower 8', 'hm_url': 'www.cjlogistics.com', 'ir_url': 'www.cjlogistics.com', 'phn_no': '02-3782-0114', 'fax_no': '02-771-8624', 'induty_code': '493', 'est_dt': '19301115', 'acc_mt': '12'}


In [62]:
prompt_template_cfo_role_current_state = """As the CFO role, please check the current financial statement and dfine the current state of the company.

context:
{context}"""
prompt_template_cfo_role_business_action = """As the CFO role, please make the Business Goals and Actions to be taken next year for the given financial statement. Define larger categories for Business Goals. Provide the response in the format of Category | Business Goal | Action.

context:
{context}"""
prompt_template_cfo_role_finance_summary = """Summary the given financial statement and provide the summary with growth rate and direction of the company. Check the overall score of this company

Context :
{context}
"""
prompt_template_cfo_role_finance_score = """As a M&A investor in a investment bank, Check the overall financial statement and provide whether the company is in good shape or not. Provide the overall score of this company from 0 to 100

Context :
{context}
"""

def makeCurrentStateAsCfo(financial_statement):
  response = llm.predict(prompt_template_cfo_role_current_state.format(context=financial_statement))
  return response

def makeBusinessGoalAsCfo(financial_statement):
  response = llm.predict(prompt_template_cfo_role_business_action.format(context=financial_statement))
  return response

def makeFinancialSummary(financial_statement):
  response = llm.predict(prompt_template_cfo_role_finance_summary.format(context=financial_statement))
  return response

def makeFinancialScore(financial_statement):
  response = llm.predict(prompt_template_cfo_role_finance_score.format(context=financial_statement))
  return response

In [ ]:
# 산업분류 코드 확인
import pandas as pd

url = 'https://github.com/FinanceData/KSIC/raw/master/KSIC_10.csv.gz'

df_ksic = pd.read_csv(url, dtype='str')
#df_ksic[df_ksic['Industy_code'] == '493']

In [ ]:
def getIndustryName(code):
  return df_ksic[df_ksic['Industy_code'] == code]['Industy_name'].values[0]

print(getIndustryName('493'))
